#### Usando randomsearch para buscar los mejores hiperparametros en xgboost

In [10]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import Imputer

import matplotlib.pyplot as plt
import utils.encodeador as my_encoder
import utils.featurizerV2 as ft_v2

import re

from math import sqrt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")


In [12]:
df_test = pd.read_csv("data/test.csv")

In [13]:
train = ft_v2.featurizerv2(train,"train")
test = ft_v2.featurizerv2(test,"test")

In [14]:
train.shape

(240000, 55)

In [15]:
test.shape

(60000, 54)

In [16]:
drop_cols = ['id','titulo','descripcion','direccion','fecha']

In [17]:
X = train.drop(['precio'] + drop_cols, axis=1)
test = test.drop(drop_cols, axis=1)
y = train['precio']

In [18]:
X.columns

Index(['tipodepropiedad', 'ciudad', 'provincia', 'antiguedad', 'habitaciones',
       'garages', 'banos', 'metroscubiertos', 'metrostotales', 'idzona', 'lat',
       'lng', 'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'seguridad', 'plazas_y_derivados',
       'patio_o_jardin', 'en_avenida', 'con_referencias_acuaticas',
       'cantidad_de_beneficios', 'con_referencias_turisticas', 'es_habitable',
       'dolar', 'devaluacion_acumulada', 'year', 'mes', 'dia_del_mes',
       'dia_del_anio', 'quarter', 'dia_de_semana', 'es_finde', 'aire_libre',
       'is_DF', 'x', 'y', 'z', 'mean_metroscubiertos_tipodepropiedad',
       'mean_metrostotales_tipodepropiedad', 'mean_gimnasio_tipodepropiedad',
       'mean_usosmultiples_tipodepropiedad', 'mean_piscina_tipodepropiedad',
       'mean_escuelascercanas_tipodepropiedad',
       'mean_centroscomercialescercanos_tipodepropiedad',
       'mean_garages_tipodepropiedad', 'mean_banos_tipodepropiedad',

In [19]:
X,test = my_encoder.oneHotEncoder(X,test)

In [20]:
X.shape

(240000, 631)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [33]:
hyper_param = { 'objective' :'reg:squarederror', 
                'colsample_bytree' : 0.3, 
               'learning_rate' : 0.1,
                'max_depth' : 15,
               'alpha' : 10,
               'n_estimators' : 300,
               'eval_metric' : 'mae'
              }

In [38]:
xgb = XGBRegressor(**hyper_param)

In [39]:
parameters = {
    'max_depth': [5, 10, 15, 20],
    #'n_estimators': [300, 500],saqué el 100 y 1000
    'min_child_weight': [1, 5, 10, 25],
    'learning_rate': [0.1, 0.01, 0.05],
    'subsample': [0.5,0.8,1],
    'colsample_bytree': [0.5,0.8,1],
    'gamma': [0.5, 1, 2, 5],
    'colsample_bytree': [0.5, 0.75, 1]
}


In [40]:
#gs = GridSearchCV(xgb, param_grid= parameters, scoring='neg_mean_absolute_error',n_jobs=3)
randomSearchXGB = RandomizedSearchCV(xgb, parameters, scoring ='neg_mean_absolute_error', n_jobs=-1, n_iter= 10, verbose= 5)

In [41]:
%%time
#gs.fit(X_train,y_train)
randomSearchXGB.fit(X_train,y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed: 75.4min
/home/braianvillalba/.local/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


KeyboardInterrupt: 

In [ ]:
gs.best_score

In [50]:
hyperOpt = {
    'objective' :'reg:squarederror', 
    'colsample_bytree' : 0.5, 
    'learning_rate' : 0.1,
    'max_depth' : 15,
    'alpha' : 10,
    'n_estimators' : 300,
    'eval_metric' : 'mae',
    'subsample' : 1,
    'min_child_weight' : 25,
    'gamma' : 5
}

In [51]:
xgb = XGBRegressor(**hyperOpt)

In [52]:
%%time
xgb.fit(X_train, y_train)

/home/braianvillalba/.local/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


CPU times: user 20min 19s, sys: 2.05 s, total: 20min 21s
Wall time: 20min 22s


XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.5, eval_metric='mae',
             gamma=5, importance_type='gain', learning_rate=0.1,
             max_delta_step=0, max_depth=15, min_child_weight=25, missing=None,
             n_estimators=300, n_jobs=1, nthread=None,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
             subsample=1, verbosity=1)

In [53]:
pred = xgb.predict(X_test)

In [54]:
mean_absolute_error(y_test,pred)

532394.063667334

In [58]:
predicion = xgb.predict(test)

In [60]:
resultado = pd.DataFrame(predicion, index=df_test.id, columns=['target'])
display(resultado.head())
resultado.to_csv("data/workshop-xgboost-v7_0.csv", header=True) 

,target
id,
4941,6704211.500
51775,1132706.250
115253,2217917.000
299321,1457579.875
173570,643518.125
